<a href="https://colab.research.google.com/github/CryptoRobotFr/General-code-trading-bot/blob/main/kucoin_sniper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ccxt

In [ ]:
import ccxt
import time

class SpotKucoin():
    def __init__(self, apiKey=None, secret=None, password=None):
        kucoinAuthObject = {
            "apiKey": apiKey,
            "secret": secret,
            "password": password,
        }
        if kucoinAuthObject['secret'] == None:
            self._auth = False
            self._session = ccxt.kucoin()
        else:
            self._auth = True
            self._session = ccxt.kucoin(kucoinAuthObject)
        self.market = self._session.load_markets()

    def reload_markets(self):
        self.market = self._session.load_markets()

    def authentication_required(fn):
        """Annotation for methods that require auth."""
        def wrapped(self, *args, **kwargs):
            if not self._auth:
                print("You must be authenticated to use this method", fn)
                exit()
            else:
                return fn(self, *args, **kwargs)
        return wrapped

    def convert_amount_to_precision(self, symbol, amount):
        return self._session.amount_to_precision(symbol, amount)

    def convert_price_to_precision(self, symbol, price):
        return self._session.price_to_precision(symbol, price)

    @authentication_required
    def place_market_order(self, symbol, side, amount):
        try:
            return self._session.createOrder(
                symbol, 
                'market', 
                side, 
                self.convert_amount_to_precision(symbol, amount),
                None
            )
        except Exception as err:
            raise err

    @authentication_required
    def place_limit_order(self, symbol, side, amount, price):
        try:
            return self._session.createOrder(
                symbol, 
                'limit', 
                side, 
                self.convert_amount_to_precision(symbol, amount), 
                self.convert_price_to_precision(symbol, price)
                )
        except Exception as err:
            raise err

kucoin = SpotKucoin(
    apiKey="",
    secret="",
    password=""
)

symbol = "BTC/USDT"
amount = 10
# buy_price = 30000
# sell_price = 35000
seconds_before_sell = 30

while True:
    try:
        kucoin.reload_markets()

        kucoin.place_market_order(symbol, "buy", amount)
        # kucoin.place_limit_order(symbol, "buy", amount, price)
        print("Buy Order success!")

        print("Waiting for sell...")
        time.sleep(seconds_before_sell)

        kucoin.place_market_order(symbol, "sell", amount)
        # kucoin.place_limit_order(symbol, "sell", amount, price)
        print("Sell Order success!")

        break
    
    except Exception as err:
        print(err)
        if str(err) == "kucoin does not have this market symbol " + symbol:
            time.sleep(0.2)
        pass